### Load packages 

In [2]:

import os
import glob

project_dir = "/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/"
#project_dir = "/standard/vol328/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/"
#project_dir ="/Users/zqn7td/Desktop/9samples_cellstate/"

os.chdir("/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/AP1_HetMel_Scripts")

In [3]:
##Import packages


import anndata as ad
import scimap as sm
import importlib

import scanpy as sp
import pandas as pd
import numpy as np
import napari as napari
import math
import seaborn as sns
import itertools

from scipy import stats
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import colors

from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster, leaves_list
from scipy.spatial.distance import pdist
from matplotlib.colors import Normalize

import hetmel as hm

Running SCIMAP  2.3.5


/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [57]:
os.chdir('/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/AP1_Hetmel_Scripts')
importlib.reload(hm)

<module 'hetmel' from '/Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/AP1_HetMel_Scripts/hetmel.py'>

## Set Case number here

Case_numbers per IMAGE ID

In [167]:
case_numbers = ['Case1-31']

In [153]:
case_numbers = [
    "Case1-31",
    "Case2-8",
    "Case3-9",
    "Case4-5_R1",
    "Case4-5_R2",
    "Case7-10",
    "Case8-9_R1",
    "Case8-9_R2",
    "Case9-7_R2",
    "Case10-3_R1",
    "Case10-3_R2",
    "Case11-13"
]

phenotype = pd.read_csv(os.path.join(project_dir, '20251002_phenotyping_workflow_fullpanel.csv'))

Case_numbers per TISSUE ID

In [203]:
case_numbers = [
    "Case1-31",
    "Case2-8",
    "Case3-9",
    "Case4-5",
    "Case7-10",
    "Case8-9",
    "Case9-7",
    "Case10-3",
    "Case11-13"
]

## Generate list containing normalized adata of all cases

Loop through adata for each case, normalize the adata to that case's gate values, and append normalized data to a list

In [11]:
adata_list = []

for case_number in case_numbers:
    if gate_file_paths[case_number] is not None:
        print(f"Reading in raw adata for {case_number}")
        # Read in raw adata file for this case
        adata_path = os.path.join(project_dir, 'raw_adata', f'{case_number}_raw.h5ad')
        adata = ad.read_h5ad(adata_path)
        
        print(f"Copying gates from {gate_file_paths[case_number]}")
        marker_gates = pd.read_csv(os.path.join( project_dir, case_number, f'{case_number}_full_gates.csv'),
        header = [0], index_col =[0])
        adata.uns['gates'] = marker_gates
        
        hm.normalize_to_gates(adata)
        # Copy the original layer
        adata.layers['log_background_normalized_zeroed'] = adata.layers['log_background_normalized'].copy()
        # Zero out negative values
        adata.layers['log_background_normalized_zeroed'][adata.layers['log_background_normalized_zeroed'] < 0] = 0
        
        adata.obs['TissueID']= adata.obs['imageid'].str.split('_').str[0]
        
        adata_list.append(adata)
    else:
        print(f"Skipping {case_number}: No gates.csv file found, cannot normalize and combine")
        


Reading in raw adata for Case1-31
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case1-31/Case1-31_full_gates.csv
Subtracting the following gate values from each marker:
  Hoechst_1: 0.0
  PRAME: 2.86
  MITF: 4.3
  JUND: 3.7
  SOX10: 4.6
  cJUN: 2.9
  cFOS: 3.9
  NGFR: 6.5
  SOX9: 4.2
  Ki67: 2.85
  CD45: 3.6
  AXL: 3.6
  FRA1: 4.9
  Hoechst_6: 0.0
  CD4: 3.9
  JUNB: 2.9
  pancK: 2.7
  FRA2: 3.0
  CD163: 4.76
  alphaSMA: 3.4
  CD8: 4.1
  CD19: 5.08
  Hoechst_11: 7.343920451148872
  CD31: 4.1
  PDGFRBeta: 4.86
  pSTAT1: 3.48
  FOXP3: 3.7
  CD68: 3.4
  BRAFV600E: 5.0
  phospho-p38: 3.7
  anti-Melanoma: 2.87
  phospho-p44: 5.71
  Hoechst_17: 7.251324716064622
  FAP: 5.21
Background normalization complete. Layer 'log_background_normalized' added.
Reading in raw adata for Case2-8


KeyboardInterrupt: 

In [22]:
adata_list= []
for case in case_numbers:
    adata_path = os.path.join(project_dir, 'raw_adata', f'{case}_raw.h5ad')
    adata = ad.read_h5ad(adata_path)
    
    adata_list.append(adata)

In [ ]:
processed_adata_list= []
for case in case_numbers:
    adata_path = os.path.join(project_dir, 'processed_adata', f'{case}_adata.h5ad')
    adata = ad.read_h5ad(adata_path)
    
    processed_adata_list.append(adata)

In [140]:
adata2=ad.read_h5ad(os.path.join(project_dir, 'processed_adata', f'Case2-8_TME_new_adata.h5ad'))

Read in adata list if necessary

In [204]:
adata_list=[]
for case in case_numbers:
    adata_path = os.path.join(project_dir, 'processed_adata', f'{case}_TME_new_adata.h5ad')
    adata = ad.read_h5ad(adata_path)
    
    
    adata_list.append(adata)

In [170]:
adata_list

[AnnData object with n_obs × n_vars = 33720 × 34
     obs: 'X_centroid', 'Y_centroid', 'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'Solidity', 'Extent', 'Orientation', 'CellID', 'imageid', 'TissueID', 'ROI'
     uns: 'all_markers', 'gates'
     layers: 'log', 'log1p_raw']

In [64]:
for adata in adata_list[2:3]:
    case_number= "Case3-9"
    print(f"Adding ROI image for {case_number}")
    image_search_pattern = os.path.join(project_dir, case_number, f'*{case_number}*fullpanel*tif')
    image_files = glob.glob(image_search_pattern, recursive=True)
    
    if image_files:
        image_path = image_files[0]
        print(f"image_path for {case_number}: {image_path}")
    else:
        image_path = None
        print(f"No image found for {case_number}")
        
    sm.pl.addROI_image(image_path, adata, markers=['Hoechst_1','MITF','SOX10','PRAME','NGFR','SOX9','cFOS','cJUN','CD45'])
    print(adata.obs['ROI'].value_counts(dropna=False))
    ROIs = adata.obs['ROI']
    print(f"Saving ROIs for {case_number}")
    ROIs.to_csv(os.path.join(project_dir, case_number,f'{case_number}_ROIs.csv'), index=True)

Adding ROI image for Case3-9
image_path for Case3-9: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/Case3-9/Case3-9_fullpanel_final.tif

        Opening Napari;
        Add shape layers (on left) to draw ROI's. 
        Rename the shape layer to give a name to your ROI
        Multiple shape layers are supported
        ROI's should not overlap
        Close Napari to save ROI's.
        


/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/napari/_vispy/layers/image.py:251: UserWarning:

data shape (22700, 25500) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/napari/_vispy/layers/image.py:251: UserWarning:

data shape (22700, 25500) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.



Identifying cells within selected ROI's
ROIs saved under adata.obs['ROI']
ROI
Tumor    504129
Other     97352
Name: count, dtype: int64
Saving ROIs for Case3-9


if you need to, load in ROI labels from csv

In [20]:
for adata in adata_list:
    case_number=str(adata.obs['imageid'].unique()[0])
    ROI_path = glob.glob(os.path.join(project_dir, case_number, '*ROIs.csv'))[0]
    ROIs= pd.read_csv(ROI_path, index_col=0)
    adata.obs['ROI'] = ROIs

IndexError: list index out of range

Saving ROIs

In [66]:
for case_number, adata in zip(case_numbers, processed_adata_list):
    ROIs = adata.obs['ROI']
    print(f"Saving ROIs for {case_number}")
    ROIs.to_csv(os.path.join(project_dir, case_number,f'{case_number}_ROIs.csv'), index=True)

Saving ROIs for Case1-31
Saving ROIs for Case2-8
Saving ROIs for Case4-5_R1
Saving ROIs for Case4-5_R2
Saving ROIs for Case7-10
Saving ROIs for Case8-9_R1
Saving ROIs for Case8-9_R2
Saving ROIs for Case9-7_R2
Saving ROIs for Case10-3_R1
Saving ROIs for Case10-3_R2
Saving ROIs for Case11-13


Reading in ROIs

In [68]:
for case_number, adata in zip(case_numbers, adata_list):
    
    ROIs= pd.read_csv(os.path.join(project_dir, case_number,f'{case_number}_ROIs.csv'), index_col=0)
    adata.obs['ROI'] = ROIs

In [205]:

gate_file_paths = {}

for case_number in case_numbers:
    # Search for gates.csv inside the case-specific subdirectory
    gate_path = os.path.join(project_dir, 'gates', f'{case_number}_full_gates.csv')
    
    if gate_path:
        gate_file_paths[case_number] = gate_path  # Take the first match
        print(f"Found for {case_number}: {gate_path}")
   

Found for Case1-31: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case1-31_full_gates.csv
Found for Case2-8: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case2-8_full_gates.csv
Found for Case3-9: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case3-9_full_gates.csv
Found for Case4-5: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case4-5_full_gates.csv
Found for Case7-10: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case7-10_full_gates.csv
Found for Case8-9: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case8-9_full_gates.csv
Found for Case9-7: /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gat

Read in gates of necessary

In [206]:
for case_number,adata in zip(case_numbers,adata_list):
    print(f"Copying gates from {gate_file_paths[case_number]}")
    marker_gates = pd.read_csv(os.path.join( project_dir, "gates", f'{case_number}_full_gates.csv'),
    header = [0], index_col =[0])
    adata.uns['gates'] = marker_gates

Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case1-31_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case2-8_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case3-9_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case4-5_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case7-10_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates/Case8-9_full_gates.csv
Copying gates from /Volumes/FallahiLab/Maize-Data/People/Kim_Nguyen/AP-1_HetMel_analysis/20250707_9samples_cellstate/gates

In [178]:
adata_list[0].uns['gates']

,markers,gates
0,AXL,3.600000
1,CD4,3.900000
2,CD45,3.600000
3,FRA1,4.900000
4,FRA2,3.000000
5,Hoechst,5.000000
6,JUNB,2.900000
7,JUND,3.700000
8,Ki67,2.850000
9,MITF,4.300000


In [189]:
adata_list[0].raw.X

array([[2271.15517241,   16.89655172,   36.0862069 , ...,   33.46551724,
        2569.13793103,  269.24      ],
       [1791.02941176,   72.23529412,   97.05882353, ...,  337.47058824,
        1718.20588235,  604.54285714],
       [1933.04545455,   59.22727273,   67.04545455, ...,  253.5       ,
        1834.22727273,  303.47169811],
       ...,
       [1899.18918919,   17.75675676,   12.08108108, ...,    7.89189189,
          39.16216216,   10.57718121],
       [1146.14566929,   60.22047244,   15.90944882, ...,   19.7480315 ,
         535.79133858,    9.39194139],
       [ 560.54216867,   27.15662651,    6.22891566, ...,    9.31325301,
        1332.71084337,    5.63120567]])

## Phenotyping

Check how many if there are NaNs in the raw data, and how many. Replace them with zeros.

In [207]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import anndata as ad

for adata in adata_list:
    case = adata.obs['imageid'].unique()[0]
    print(f"\nChecking {case} for NaNs")

    # --- Extract raw matrix as dense ---
    if adata.raw is not None:
        raw_X = adata.raw.X.A if hasattr(adata.raw.X, "A") else adata.raw.X
    else:
        print("No .raw found, skipping raw check.")
        raw_X = None

    # --- Identify NaNs (only if raw exists) ---
    if raw_X is not None:
        data_df = pd.DataFrame(
            raw_X,
            columns=adata.raw.var_names,
            index=adata.obs_names
        )
        nan_counts = data_df.isna().sum().sort_values(ascending=False)
        if nan_counts.sum() == 0:
            print("No NaNs found in raw.X, skipping replacement.")
            continue

        print("Markers with NaNs in raw.X:")
        print(nan_counts[nan_counts > 0])
    else:
        nan_counts = pd.Series(dtype=float)

    print("Replacing NaNs with zeros...")

    # --- Replace NaNs in main matrix ---
    if sp.issparse(adata.X):
        adata.X = adata.X.toarray()
    adata.X = np.nan_to_num(adata.X, nan=0)

    # --- Replace NaNs in raw.X (must rebuild) ---
    if raw_X is not None:
        cleaned_raw_X = np.nan_to_num(raw_X, nan=0)
        adata.raw = ad.AnnData(
            X=cleaned_raw_X,
            var=adata.raw.var.copy(),
            obs=adata.obs.copy()  # use parent obs
        )

    print(f"NaN replacement complete for {case}.")



Checking Case1-31 for NaNs
Markers with NaNs in raw.X:
FAP              2
PDGFRBeta        2
pancK            2
anti-Melanoma    2
BRAFV600E        2
NGFR             2
CD68             2
alphaSMA         2
CD31             2
AXL              2
dtype: int64
Replacing NaNs with zeros...
NaN replacement complete for Case1-31.

Checking Case2-8 for NaNs
Markers with NaNs in raw.X:
anti-Melanoma    75
pancK            75
BRAFV600E        75
CD68             75
NGFR             75
FAP              75
alphaSMA         75
PDGFRBeta        75
AXL              75
CD31             75
dtype: int64
Replacing NaNs with zeros...
NaN replacement complete for Case2-8.

Checking Case3-9 for NaNs
Markers with NaNs in raw.X:
FAP              7
PDGFRBeta        7
pancK            7
anti-Melanoma    7
BRAFV600E        7
NGFR             7
CD68             7
alphaSMA         7
CD31             7
AXL              7
dtype: int64
Replacing NaNs with zeros...
NaN replacement complete for Case3-9.

Checking Cas

Go through adata.uns['gates'] and make column names uniformly "markers" and "gates"

In [209]:
import pandas as pd
import scimap as sm

for i, adata in enumerate(adata_list):
    if 'gates' not in adata.uns:
        print(f"adata_list[{i}] has no 'gates', skipping.")
        continue

    gates_data = adata.uns['gates']

    # If DataFrame with markers as index and a single numeric column
    if isinstance(gates_data, pd.DataFrame):
        # Take first numeric column as 'gates'
        numeric_cols = gates_data.select_dtypes(include='number').columns
        if len(numeric_cols) == 0:
            print(f"adata_list[{i}]: No numeric column found, skipping.")
            continue

        gates_df = gates_data[numeric_cols[0]].reset_index()
        gates_df.columns = ['markers', 'gates']

        # Assign back
        adata.uns['gates'] = gates_df


print("All adata objects processed and gates columns fixed.")


All adata objects processed and gates columns fixed.


Log transform and rescale data in adata.raw.x such that the gate value for each marker is 0.5. Values above 0.5 will be phenotyped + and values below 0.5 will be phenotyped -

In [212]:
for adata in adata_list[1:]:
    sm.pp.rescale(adata, gate=adata.uns['gates'])
    #adata = sm.tl.phenotype_cells(adata, phenotype=phenotype, label="phenotype")

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/preprocessing/rescale.py:126: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`




Scaling Image: Case2-8
Scaling Hoechst_1 (gate: 8.064)
Scaling PRAME (gate: 3.400)
Scaling MITF (gate: 3.280)
Scaling JUND (gate: 4.100)
Scaling SOX10 (gate: 5.540)
Scaling cJUN (gate: 4.060)
Scaling cFOS (gate: 4.290)
Scaling NGFR (gate: 5.420)
Scaling SOX9 (gate: 4.110)
Scaling Ki67 (gate: 4.580)
Scaling CD45 (gate: 3.850)
Scaling AXL (gate: 5.540)
Scaling FRA1 (gate: 4.080)
Scaling Hoechst_6 (gate: 7.749)
Scaling JUNB (gate: 3.920)
Scaling CD4 (gate: 4.270)
Scaling pancK (gate: 3.700)
Scaling FRA2 (gate: 4.370)
Scaling CD163 (gate: 4.700)
Scaling alphaSMA (gate: 5.400)
Scaling CD8 (gate: 4.500)
Scaling CD19 (gate: 5.000)
Scaling FAP (gate: 4.800)
Scaling Hoechst_11 (gate: 7.184)
Scaling CD31 (gate: 4.190)
Scaling PDGFRBeta (gate: 4.770)
Scaling pSTAT1 (gate: 3.100)
Scaling FOXP3 (gate: 3.400)
Scaling CD68 (gate: 3.860)
Scaling BRAFV600E (gate: 3.000)
Scaling phospho-p38 (gate: 3.800)
Scaling Hoechst_16 (gate: 7.243)
Scaling anti-Melanoma (gate: 3.700)
Scaling phospho-p44 (gate: 3.5

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/preprocessing/rescale.py:126: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`




Scaling Image: Case3-9
Scaling Hoechst_1 (gate: 7.766)
Scaling PRAME (gate: 3.350)
Scaling MITF (gate: 3.300)
Scaling JUND (gate: 3.750)
Scaling SOX10 (gate: 5.500)
Scaling cJUN (gate: 3.170)
Scaling cFOS (gate: 3.600)
Scaling NGFR (gate: 5.250)
Scaling SOX9 (gate: 4.200)
Scaling Ki67 (gate: 3.300)
Scaling CD45 (gate: 3.600)
Scaling AXL (gate: 3.270)
Scaling FRA1 (gate: 3.930)
Scaling Hoechst_6 (gate: 7.043)
Scaling CD4 (gate: 4.300)
Scaling JUNB (gate: 3.180)
Scaling pancK (gate: 2.460)
Scaling FRA2 (gate: 3.490)
Scaling CD163 (gate: 3.850)
Scaling alphaSMA (gate: 3.760)
Scaling CD8 (gate: 4.670)
Scaling CD19 (gate: 4.200)
Scaling Hoechst_11 (gate: 6.703)
Scaling CD31 (gate: 4.310)
Scaling PDGFRBeta (gate: 4.300)
Scaling pSTAT1 (gate: 3.530)
Scaling FOXP3 (gate: 3.560)
Scaling CD68 (gate: 3.870)
Scaling BRAFV600E (gate: 2.800)
Scaling phospho-p38 (gate: 3.570)
Scaling anti-Melanoma (gate: 3.590)
Scaling phospho-p44 (gate: 4.930)
Scaling Hoechst_17 (gate: 6.853)
Scaling FAP (gate: 6.0

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/preprocessing/rescale.py:126: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



...

Running GMM for image: Case9-7
Applying GMM to markers: Hoechst_1, Hoechst_6

Scaling Image: Case9-7
Scaling Hoechst_1 (gate: 7.451)
Scaling PRAME (gate: 2.910)
Scaling MITF (gate: 3.700)
Scaling JUND (gate: 3.640)
Scaling SOX10 (gate: 4.970)
Scaling cJUN (gate: 2.740)
Scaling cFOS (gate: 4.080)
Scaling NGFR (gate: 4.567)
Scaling SOX9 (gate: 3.300)
Scaling Ki67 (gate: 3.360)
Scaling CD45 (gate: 3.630)
Scaling AXL (gate: 3.600)
Scaling FRA1 (gate: 4.200)
Scaling Hoechst_6 (gate: 7.624)
Scaling CD4 (gate: 4.240)
Scaling JUNB (gate: 3.320)
Scaling pancK (gate: 3.060)
Scaling FRA2 (gate: 3.270)
Scaling CD163 (gate: 3.670)
Scaling alphaSMA (gate: 4.000)
Scaling CD8 (gate: 4.250)
Scaling CD19 (gate: 4.100)
Scaling Hoechst_11 (gate: 7.035)
Scaling CD31 (gate: 4.020)
Scaling PDGFRBeta (gate: 4.360)
Scaling pSTAT1 (gate: 3.100)
Scaling FOXP3 (gate: 3.350)
Scaling CD68 (gate: 4.400)
Scaling BRAFV600E (gate: 2.766)
Scaling phospho-p38 (gate: 3.600)
Scaling anti-Melanoma (gate: 5.210)
Scaling phos

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/preprocessing/rescale.py:126: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



Running GMM for image: Case10-3
Applying GMM to markers: Hoechst_1, Hoechst_6

Scaling Image: Case10-3
Scaling Hoechst_1 (gate: 6.899)
Scaling PRAME (gate: 3.450)
Scaling MITF (gate: 3.330)
Scaling JUND (gate: 3.490)
Scaling SOX10 (gate: 5.400)
Scaling cJUN (gate: 3.200)
Scaling cFOS (gate: 3.220)
Scaling NGFR (gate: 6.230)
Scaling SOX9 (gate: 3.800)
Scaling Ki67 (gate: 5.360)
Scaling CD45 (gate: 4.450)
Scaling AXL (gate: 5.000)
Scaling FRA1 (gate: 4.870)
Scaling Hoechst_6 (gate: 7.052)
Scaling JUNB (gate: 4.700)
Scaling CD4 (gate: 4.180)
Scaling pancK (gate: 3.440)
Scaling FRA2 (gate: 4.050)
Scaling CD163 (gate: 4.200)
Scaling alphaSMA (gate: 3.000)
Scaling CD8 (gate: 3.800)
Scaling CD19 (gate: 4.600)
Scaling FAP (gate: 4.500)
Scaling Hoechst_11 (gate: 6.768)
Scaling CD31 (gate: 3.500)
Scaling PDGFRBeta (gate: 4.000)
Scaling pSTAT1 (gate: 3.900)
Scaling FOXP3 (gate: 3.500)
Scaling CD68 (gate: 3.960)
Scaling BRAFV600E (gate: 3.078)
Scaling phospho-p38 (gate: 4.050)
Scaling Hoechst_16 (

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/preprocessing/rescale.py:126: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



Running GMM for image: Case11-13
Applying GMM to markers: Hoechst_1, Hoechst_6

Scaling Image: Case11-13
Scaling Hoechst_1 (gate: 7.601)
Scaling PRAME (gate: 2.960)
Scaling MITF (gate: 4.000)
Scaling JUND (gate: 3.230)
Scaling SOX10 (gate: 5.220)
Scaling cJUN (gate: 3.340)
Scaling cFOS (gate: 5.000)
Scaling NGFR (gate: 6.200)
Scaling SOX9 (gate: 4.500)
Scaling Ki67 (gate: 3.130)
Scaling CD45 (gate: 3.270)
Scaling AXL (gate: 3.960)
Scaling FRA1 (gate: 5.000)
Scaling Hoechst_6 (gate: 7.596)
Scaling CD4 (gate: 3.530)
Scaling JUNB (gate: 3.250)
Scaling pancK (gate: 4.619)
Scaling FRA2 (gate: 3.710)
Scaling CD163 (gate: 4.200)
Scaling alphaSMA (gate: 4.000)
Scaling CD8 (gate: 4.900)
Scaling CD19 (gate: 4.600)
Scaling Hoechst_11 (gate: 6.838)
Scaling CD31 (gate: 4.210)
Scaling PDGFRBeta (gate: 4.050)
Scaling pSTAT1 (gate: 3.820)
Scaling FOXP3 (gate: 3.200)
Scaling CD68 (gate: 3.800)
Scaling BRAFV600E (gate: 2.846)
Scaling phospho-p38 (gate: 3.960)
Scaling anti-Melanoma (gate: 4.300)
Scaling 

Run phenotyping on cells using the phenotyping workflow. Phenotypes with prevalance below 0.1% are labeled unknown 

In [234]:
for adata in adata_list:
    adata = sm.tl.phenotype_cells(adata, phenotype=phenotype, label="phenotype", pheno_threshold_percent=0.1)

Phenotyping Tumor cell
Phenotyping Immune cell
Phenotyping Fibroblast
Phenotyping Endothelial cell
Phenotyping Keratinocyte
-- Subsetting Immune cell
Phenotyping Cytotoxic T cell
Phenotyping Helper T cell
Phenotyping B cell
Phenotyping Macrophage
-- Subsetting Helper T cell
Phenotyping T-Regulatory cell
-- Subsetting Macrophage
Phenotyping Pro-tumor macrophage
Phenotyping Inflammatory macrophage
-- Subsetting Tumor cell
Phenotyping MITF+ SOX10+ PRAME+ NGFR+ SOX9+ AXL-


/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

Phenotyping MITF+ SOX10+ PRAME- NGFR+ SOX9+ AXL-
Phenotyping MITF+ SOX10+ PRAME+ NGFR+ SOX9- AXL-
Phenotyping MITF+ SOX10+ PRAME+ NGFR- SOX9+ AXL-
Phenotyping MITF+ SOX10- PRAME+ NGFR+ SOX9+ AXL-
Phenotyping MITF- SOX10+ PRAME+ NGFR+ SOX9+ AXL-
Phenotyping MITF+ SOX10+ PRAME- NGFR+ SOX9- AXL-
Phenotyping MITF+ SOX10+ PRAME- NGFR- SOX9+ AXL-
Phenotyping MITF+ SOX10+ PRAME+ NGFR- SOX9- AXL-
Phenotyping MITF+ SOX10- PRAME- NGFR+ SOX9+ AXL-
Phenotyping MITF+ SOX10- PRAME+ NGFR+ SOX9- AXL-
Phenotyping MITF+ SOX10- PRAME+ NGFR- SOX9+ AXL-
Phenotyping MITF- SOX10+ PRAME- NGFR+ SOX9+ AXL-
Phenotyping MITF- SOX10+ PRAME+ NGFR+ SOX9- AXL-
Phenotyping MITF- SOX10+ PRAME+ NGFR- SOX9+ AXL-
Phenotyping MITF- SOX10- PRAME+ NGFR+ SOX9+ AXL-
Phenotyping MITF+ SOX10+ PRAME- NGFR- SOX9- AXL-
Phenotyping MITF+ SOX10- PRAME- NGFR+ SOX9- AXL-
Phenotyping MITF+ SOX10- PRAME- NGFR- SOX9+ AXL-
Phenotyping MITF+ SOX10- PRAME+ NGFR- SOX9- AXL-
Phenotyping MITF- SOX10+ PRAME- NGFR+ SOX9- AXL-
Phenotyping MITF- SO

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

Phenotyping MITF- SOX10+ PRAME- NGFR- SOX9- AXL+
Phenotyping MITF- SOX10- PRAME- NGFR+ SOX9- AXL+
Phenotyping MITF- SOX10- PRAME- NGFR- SOX9+ AXL+
Phenotyping MITF- SOX10- PRAME+ NGFR- SOX9- AXL+
Phenotyping MITF+ SOX10+ PRAME- NGFR- SOX9- AXL+
Phenotyping MITF+ SOX10- PRAME- NGFR+ SOX9- AXL+
Phenotyping MITF+ SOX10- PRAME- NGFR- SOX9+ AXL+
Phenotyping MITF+ SOX10- PRAME+ NGFR- SOX9- AXL+
Phenotyping MITF- SOX10+ PRAME- NGFR+ SOX9- AXL+
Phenotyping MITF- SOX10+ PRAME- NGFR- SOX9+ AXL+
Phenotyping MITF- SOX10+ PRAME+ NGFR- SOX9- AXL+
Phenotyping MITF- SOX10- PRAME- NGFR+ SOX9+ AXL+
Phenotyping MITF- SOX10- PRAME+ NGFR+ SOX9- AXL+
Phenotyping MITF- SOX10- PRAME+ NGFR- SOX9+ AXL+
Phenotyping MITF+ SOX10+ PRAME- NGFR+ SOX9+ AXL+
Phenotyping MITF+ SOX10+ PRAME- NGFR- SOX9+ AXL+
Phenotyping MITF+ SOX10+ PRAME+ NGFR- SOX9+ AXL+
Phenotyping MITF+ SOX10- PRAME- NGFR+ SOX9+ AXL+
Phenotyping MITF+ SOX10- PRAME+ NGFR+ SOX9- AXL+
Phenotyping MITF+ SOX10- PRAME+ NGFR- SOX9+ AXL+
Phenotyping MITF- SO

...

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

Phenotyping MITF- SOX10- PRAME+ NGFR+ SOX9+ AXL+
Phenotyping MITF- SOX10- PRAME- NGFR- SOX9- AXL+
Phenotyping MITF+ SOX10+ PRAME+ NGFR+ SOX9+ AXL+
Phenotyping MITF+ SOX10+ PRAME+ NGFR+ SOX9- AXL+
Phenotyping MITF+ SOX10- PRAME+ NGFR+ SOX9+ AXL+
Phenotyping MITF- SOX10+ PRAME+ NGFR+ SOX9+ AXL+


/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:286: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain

Consolidating the phenotypes across all groups


/Users/zqn7td/anaconda3/envs/scimap/lib/python3.10/site-packages/scimap/tools/phenotype_cells.py:290: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [235]:
adata_list[0].obs['phenotype'].value_counts()

phenotype
Fibroblast                              6721
Tumor cell                              4721
MITF- SOX10- PRAME+ NGFR- SOX9- AXL-    3784
MITF- SOX10+ PRAME+ NGFR- SOX9+ AXL-    3601
MITF+ SOX10+ PRAME+ NGFR- SOX9+ AXL-    2537
Unknown                                 2293
MITF- SOX10+ PRAME- NGFR- SOX9+ AXL-    1902
Keratinocyte                            1871
MITF- SOX10- PRAME+ NGFR- SOX9+ AXL-    1165
MITF- SOX10- PRAME+ NGFR- SOX9- AXL+     943
MITF- SOX10+ PRAME+ NGFR- SOX9- AXL-     703
MITF+ SOX10+ PRAME- NGFR- SOX9+ AXL-     650
MITF- SOX10+ PRAME- NGFR- SOX9- AXL-     559
Immune cell                              289
MITF- SOX10- PRAME+ NGFR+ SOX9- AXL-     233
MITF- SOX10+ PRAME+ NGFR+ SOX9+ AXL-     220
MITF+ SOX10+ PRAME+ NGFR+ SOX9+ AXL-     219
MITF- SOX10- PRAME+ NGFR- SOX9+ AXL+     204
Helper T cell                            187
MITF- SOX10- PRAME+ NGFR+ SOX9- AXL+     133
MITF- SOX10+ PRAME+ NGFR- SOX9+ AXL+     100
Cytotoxic T cell                          89


In [233]:
print(adata_list[4].obs['phenotype'].value_counts())

phenotype
Fibroblast                              51941
MITF+ SOX10+ PRAME+ NGFR- SOX9+ AXL-    45714
MITF+ SOX10+ PRAME- NGFR- SOX9+ AXL-    35437
Unknown                                 30567
MITF+ SOX10+ PRAME+ NGFR+ SOX9+ AXL-    26678
MITF+ SOX10+ PRAME- NGFR+ SOX9+ AXL-    18208
Keratinocyte                            10658
Tumor cell                               8638
MITF- SOX10+ PRAME- NGFR- SOX9+ AXL-     7028
MITF+ SOX10- PRAME- NGFR- SOX9+ AXL-     3625
Helper T cell                            3385
MITF- SOX10+ PRAME- NGFR+ SOX9+ AXL-     2976
MITF- SOX10+ PRAME- NGFR- SOX9- AXL-     2595
MITF+ SOX10- PRAME- NGFR- SOX9- AXL-     1884
MITF+ SOX10+ PRAME- NGFR- SOX9- AXL-     1289
Immune cell                              1217
MITF+ SOX10- PRAME+ NGFR- SOX9+ AXL-     1033
MITF+ SOX10- PRAME- NGFR+ SOX9+ AXL-     1015
MITF+ SOX10- PRAME- NGFR+ SOX9+ AXL+      830
MITF- SOX10- PRAME+ NGFR- SOX9+ AXL-      796
MITF- SOX10+ PRAME+ NGFR- SOX9+ AXL-      793
MITF- SOX10- PRAME+ NGFR

Replace all "Tumor cell" phenotypes with "Unknown"

In [236]:
for i in range(len(adata_list)):
    adata = adata_list[i]
    adata.obs['phenotype'] = adata.obs['phenotype'].replace('Tumor cell', 'Unknown')


Assign cell phenotype category. ie. Identify tumor cells vs TME 

In [239]:
for i in range(len(adata_list)):
    adata = adata_list[i]
    adata.obs['phenotype_category'] = adata.obs['phenotype'].apply(hm.categorize_phenotype)

In [242]:
adata_list[1].obs['phenotype_category'].value_counts()

phenotype_category
Tumor    287534
TME      196196
Other    191071
Name: count, dtype: int64

Normalize markers to gates

In [243]:
for adata in adata_list:
    hm.normalize_to_gates(adata)
            # Copy the original layer
    adata.layers['log_background_normalized_zeroed'] = adata.layers['log_background_normalized'].copy()
            # Zero out negative values
    adata.layers['log_background_normalized_zeroed'][adata.layers['log_background_normalized_zeroed'] < 0] = 0

Subtracting the following gate values from each marker:
  Hoechst_1: 7.7531009156742385
  PRAME: 2.86
  MITF: 4.3
  JUND: 3.7
  SOX10: 4.6
  cJUN: 2.9
  cFOS: 3.9
  NGFR: 6.5
  SOX9: 4.2
  Ki67: 2.85
  CD45: 3.6
  AXL: 3.6
  FRA1: 4.9
  Hoechst_6: 7.733164460690223
  CD4: 3.9
  JUNB: 2.9
  pancK: 2.7
  FRA2: 3.0
  CD163: 4.76
  alphaSMA: 3.4
  CD8: 4.1
  CD19: 5.08
  Hoechst_11: 7.343920451148872
  CD31: 4.1
  PDGFRBeta: 4.86
  pSTAT1: 3.48
  FOXP3: 3.7
  CD68: 3.4
  BRAFV600E: 5.0
  phospho-p38: 3.7
  anti-Melanoma: 2.87
  phospho-p44: 5.71
  Hoechst_17: 7.251324716064622
  FAP: 5.21
Background normalization complete. Layer 'log_background_normalized' added.
Subtracting the following gate values from each marker:
  Hoechst_1: 8.064459490587334
  PRAME: 3.4
  MITF: 3.28
  JUND: 4.1
  SOX10: 5.54
  cJUN: 4.06
  cFOS: 4.29
  NGFR: 5.42
  SOX9: 4.11
  Ki67: 4.58
  CD45: 3.85
  AXL: 5.54
  FRA1: 4.08
  Hoechst_6: 7.748817289428954
  JUNB: 3.92
  CD4: 4.27
  pancK: 3.7
  FRA2: 4.37
  CD163:

Save individual adata

In [221]:
for adata in adata_list:
    adata.write_h5ad(os.path.join(project_dir,'processed_adata',f"{adata.obs['imageid'].unique()[0]}_TME_adata_v2.h5ad"))